In [77]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import arff
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler,MaxAbsScaler
from scipy.stats import norm
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score, auc, roc_curve
from sklearn.metrics import make_scorer
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.decomposition import PCA
import os

In [78]:
path_to_directory="SOFTLAB/"
files = [arff for arff in os.listdir(path_to_directory) if arff.endswith(".arff")]

def toCsv(content): 
    data = False
    header = ""
    newContent = []
    for line in content:
        if not data:
            if "@attribute" in line:
                attri = line.split()
                columnName = attri[attri.index("@attribute")+1]
                header = header + columnName + ","
            elif "@data" in line:
                data = True
                header = header[:-1]
                header += '\n'
                newContent.append(header)
        else:
            newContent.append(line)
    return newContent

In [79]:
# Main loop for reading and writing files
for zzzz,file in enumerate(files):
    with open(path_to_directory+file , "r") as inFile:
        content = inFile.readlines()
        name,ext = os.path.splitext(inFile.name)
        new = toCsv(content)
        with open(name+".csv", "w") as outFile:
            outFile.writelines(new)

In [80]:
ar1 = pd.read_csv('SOFTLAB/ar1.csv')
ar3 = pd.read_csv('SOFTLAB/ar3.csv')
ar4 = pd.read_csv('SOFTLAB/ar4.csv')
ar5 = pd.read_csv('SOFTLAB/ar5.csv')
ar6 = pd.read_csv('SOFTLAB/ar6.csv')

In [81]:
list_softlab = ['ar1','ar3','ar4','ar5','ar6']
list_data_softlab = [ar1,ar3,ar4,ar5,ar6]

In [82]:
ar1

,total_loc,blank_loc,comment_loc,code_and_comment_loc,executable_loc,unique_operands,unique_operators,total_operands,total_operators,halstead_vocabulary,...,condition_count,multiple_condition_count,cyclomatic_complexity,cyclomatic_density,decision_density,design_complexity,design_density,normalized_cyclomatic_complexity,formal_parameters,defects
0,7,0,4,0,3,8,6,10,12,14,...,0,0,1,0.33,0.00,0,0.00,0.14,0,clean
1,9,0,1,0,8,7,8,15,20,15,...,0,0,2,0.25,0.00,0,0.00,0.22,0,clean
2,21,0,14,1,7,15,12,21,36,27,...,4,1,4,0.57,1.00,1,0.25,0.19,0,clean
3,30,0,11,0,19,16,18,50,70,34,...,9,2,9,0.47,1.11,1,0.11,0.30,0,buggy
4,8,0,2,0,6,4,5,5,10,9,...,1,0,2,0.33,1.00,1,0.50,0.25,0,clean
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,32,0,5,0,27,16,11,33,56,27,...,9,1,8,0.30,1.00,0,0.00,0.25,0,clean
117,6,0,0,0,6,3,5,5,11,8,...,2,0,3,0.50,1.00,0,0.00,0.50,0,clean
118,6,0,0,0,6,6,6,6,11,12,...,2,1,2,0.33,1.00,1,0.50,0.33,0,clean
119,27,0,3,0,24,16,9,26,44,25,...,8,3,6,0.25,1.00,1,0.17,0.22,0,clean


In [83]:
for dt in list_data_softlab:
    dt['Defective'] = dt['defects']
    dt.loc[dt['Defective'] == 'clean', 'Defective'] = 'N'
    dt.loc[dt['Defective'] == 'buggy', 'Defective'] = 'Y'
    dt.drop('defects',axis=1,inplace=True)

In [84]:
list_dataset = [ar1,ar4,ar3,ar6]
label_dataset = ['ar1','ar3']

list_test = [ar1,ar3,ar4,ar5,ar6]  
label_test = ['ar1','ar3','ar4','ar5','ar6'] 

my_index = range(0,len(list_test))
my_dataset = dict(zip(label_test,list_test))

df = list_dataset
df = pd.concat(df)
df

,total_loc,blank_loc,comment_loc,code_and_comment_loc,executable_loc,unique_operands,unique_operators,total_operands,total_operators,halstead_vocabulary,...,condition_count,multiple_condition_count,cyclomatic_complexity,cyclomatic_density,decision_density,design_complexity,design_density,normalized_cyclomatic_complexity,formal_parameters,Defective
0,7,0,4,0,3,8,6,10,12,14,...,0,0,1,0.33,0.00,0.0,0.00,0.14,0.0,N
1,9,0,1,0,8,7,8,15,20,15,...,0,0,2,0.25,0.00,0.0,0.00,0.22,0.0,N
2,21,0,14,1,7,15,12,21,36,27,...,4,1,4,0.57,1.00,1.0,0.25,0.19,0.0,N
3,30,0,11,0,19,16,18,50,70,34,...,9,2,9,0.47,1.11,1.0,0.11,0.30,0.0,Y
4,8,0,2,0,6,4,5,5,10,9,...,1,0,2,0.33,1.00,1.0,0.50,0.25,0.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,27,0,10,0,17,15,9,36,48,24,...,4,1,4,0.24,1.00,5.0,1.25,0.15,0.0,N
97,9,0,0,0,9,7,3,8,15,10,...,0,0,1,0.11,0.00,6.0,6.00,0.11,0.0,N
98,10,0,7,0,3,7,3,7,15,10,...,0,0,1,0.33,0.00,6.0,6.00,0.10,0.0,N
99,51,2,17,0,32,26,9,44,69,35,...,6,0,7,0.22,1.00,8.0,1.14,0.14,0.0,N


In [85]:
def adjustdataset(dt,dataframe):
    dt_copy = dt.copy()
    counter = 0
    
    for col in dataframe.columns:
        if not col in dt.columns:
            #print("Not Exist ",col)
            dt_copy.insert(counter,col,-1)
        counter += 1

    return pd.concat([dt_copy[dataframe.columns], dataframe])
    

In [86]:
def adjustdataset_list(list_df):
    dt_result = list_df.pop()
    
    for dt in list_df:
        if len(dt.columns) > len(dt_result.columns):
            dt_result = adjustdataset(dt,dt_result)
        else:
            dt_result = adjustdataset(dt_result,dt)    
    return dt_result

In [87]:
def show_distribution(df, columns):
    
    counter = 0
    # Create a figure and a grid of subplots with 4 rows and 10 columns
    fig, axes = plt.subplots(5, 8, figsize=(20, 8))
    
    # Loop through each subplot and add content (optional)
    for i in range(5):
        for j in range(8):
            try:
                ax = axes[i, j]
                ax.set_title(columns[counter])
                ax.plot(df[columns[counter]].sort_values(), norm.pdf(df[columns[counter]].sort_values(), df[columns[counter]].mean(), df[columns[counter]].std())) 
                ax.set_xticks([])
                ax.set_yticks([])
                counter += 1
            except:
                pass

# Adjust layout to prevent overlapping titles/labels
plt.tight_layout()

# Display the plot
plt.show()

<Figure size 640x480 with 0 Axes>

In [88]:
def preprocessing(dataframe, imputer = None, pt = None, scaler = None, show = True):
    
    df = dataframe.copy()
    
    df.loc[(df['Defective'] == 'N'),'Defective']=0
    df.loc[(df['Defective'] == 'Y'),'Defective']=1
    
    change_dtypes = ['Defective']
    dict_dtypes = {d:'int64' for d in change_dtypes }
    df = df.astype(dict_dtypes)
    
    columns = df.columns
    
    if show:
        show_distribution(df,df.columns)
    
    if imputer :
        df = imputer.transform(df)
    else:
        imputer = KNNImputer(n_neighbors=2)
        df = imputer.fit_transform(df)
    
    df = pd.DataFrame(df)
    df.columns = columns
    
    if show:
        show_distribution(df,df.columns)
    
    y = df['Defective']
    X = df.drop('Defective',axis=1)
    
    if pt:
        X_norm = pt.transform(X)
    else:
        pt = PowerTransformer()
        X_norm = pt.fit_transform(X)
    
    X_norm = pd.DataFrame(X)
    X_norm.columns = X.columns
    
    if show:
        show_distribution(X_norm,X_norm.columns)

    if scaler:
        X_norm_scale = scaler.transform(X_norm)
    else:
        scaler = StandardScaler()
        X_norm_scale = scaler.fit_transform(X_norm)
        
    return pt, scaler, imputer, X_norm_scale, y



In [89]:
def heat_map(y,result,title):
    import seaborn as sns
    from sklearn.metrics import confusion_matrix
    sns.set
    _, ax = plt.subplots(figsize=(10,10))
    ax = sns.heatmap(confusion_matrix(y, result), annot=True, fmt='d', cmap= ['#09acec','#089dd9','#078fc5','#0681b1','#05668d','#05368d'], annot_kws={"size": 40, "weight": "bold"})  
    labels = ['No Defect','Defect']
    ax.set_xticklabels(labels, fontsize=30);
    ax.set_yticklabels(labels, fontsize=30,rotation=0);
    ax.set_ylabel('Prediction', fontsize=30,rotation=0);
    ax.set_xlabel('Ground Truth', fontsize=30) #0,1
    ax.set_title(title)

In [90]:
def score(y,result):
    accuracy = accuracy_score(y, result)
    f1 = f1_score(y, result, average='weighted')
    precision = precision_score(y, result, average='weighted')
    recall = recall_score(y, result, average='weighted')
    print("Accuracy = " , accuracy, " Precision = ",precision, " Recall = ",recall, " F1-Score = ",f1 )

In [91]:
from sklearn.metrics import make_scorer

scorers = {
            'f1_score': make_scorer(f1_score, average='micro'),
            'precision_score': make_scorer(precision_score, average='micro'),
            'recall_score': make_scorer(recall_score, average='micro'),
            'accuracy_score': make_scorer(accuracy_score)
          }
search_space={}

In [92]:
pt, scaler, imputer, X_norm_scale, y = preprocessing(df, show=False)
#X_norm_scale_smote, y_smote = X_norm_scale, y
smote = SMOTE(random_state=42)
X_norm_scale_smote, y_smote = smote.fit_resample(X_norm_scale, y)  

/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [93]:
counter = 0
histories = []

lst_accu_stratified = []
lst_f1_stratified = []
lst_precision_stratified = []
lst_recall_stratified = []
lst_auc_stratified = []

lst_accu_stratified_val = []
lst_f1_stratified_val = []
lst_precision_stratified_val = []
lst_recall_stratified_val = []
lst_auc_stratified_val = []

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=19)

for train_index, test_index in skf.split(X_norm_scale_smote, y_smote):

    x_train_fold, x_test_fold = X_norm_scale_smote[train_index], X_norm_scale_smote[test_index]
    y_train_fold, y_test_fold = y_smote[train_index], y_smote[test_index]
 
    rte = RandomTreesEmbedding(n_estimators=1000, random_state=0, max_depth=10, sparse_output=True).fit(x_train_fold)
    x_sparse_embedding_train = rte.transform(x_train_fold)
    x_sparse_embedding_test = rte.transform(x_test_fold)
    
    with open('rte-tree-softlab-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(rte, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    from sklearn.manifold import Isomap
    manifold = Isomap(n_components=10)
    x_sparse_manifold_train = manifold.fit_transform(x_sparse_embedding_train)
    x_sparse_manifold_test = manifold.transform(x_sparse_embedding_test)
    
    with open('manifold-tree-softlab-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(manifold, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    scaler2 = StandardScaler()
    x_sparse_manifold_train = scaler2.fit_transform(x_sparse_manifold_train)
    x_sparse_manifold_test = scaler2.transform(x_sparse_manifold_test)
    
    with open('scaler-tree-softlab-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(scaler2, handle, protocol=pickle.HIGHEST_PROTOCOL)

    input_X_train1 = layers.Input(
    shape=(x_sparse_manifold_train.shape[1],), name="X_train1")  
    
    input_X_train2 = layers.Input(shape=(x_train_fold.shape[1],), name="X_train2")

    layer_X_train1 = layers.Dense(64,activation='relu',name="X_train1_layer")(input_X_train1)
    layer_X_train1 = layers.BatchNormalization()(layer_X_train1)
    layer_X_train1 = layers.Dropout(0.3)(layer_X_train1)
    
    layer_X_train2 = layers.Dense(64,activation='relu',name="X_train2_layer")(input_X_train2)
    layer_X_train2 = layers.BatchNormalization()(layer_X_train2)
    layer_X_train2 = layers.Dropout(0.3)(layer_X_train2)
    
    # Merge all available features into a single large vector via concatenation
    concat = layers.concatenate([layer_X_train1, layer_X_train2])

    layer_final = layers.Dense(1, activation='sigmoid', name="classifier")(concat)
    sdp_classifier = Model(inputs=[input_X_train1, input_X_train2], outputs=layer_final)

    checkpointer=ModelCheckpoint(filepath='sdp-tree-softlab-{0}.weights.best.keras'.format(counter),verbose=1,save_best_only=True)

    sdp_classifier.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'] )
    
    hist = sdp_classifier.fit([x_sparse_manifold_train,x_train_fold], y_train_fold, 
                    batch_size=32, epochs=100, validation_split= 0.1, callbacks=[checkpointer], 
                    verbose=2, shuffle=True)
    
    with open('sdp-classifier-tree-softlab-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(sdp_classifier, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

    sdp_classifier.load_weights('sdp-tree-softlab-{0}.weights.best.keras'.format(counter))

    histories.append(hist)

    results = sdp_classifier.predict([x_sparse_manifold_train,x_train_fold])
    results = np.round(results)
    
    lst_accu_stratified.append(accuracy_score(y_train_fold, results))
    lst_f1_stratified.append(f1_score(y_train_fold, results, average='weighted') )
    lst_precision_stratified.append(precision_score(results, y_train_fold, average='weighted') )
    lst_recall_stratified.append(recall_score(results, y_train_fold, average='weighted') )
    fpr, tpr, thresholds = roc_curve(y_train_fold, results)
    lst_auc_stratified.append(auc(fpr, tpr))
    
    results = sdp_classifier.predict([x_sparse_manifold_test,x_test_fold])
    results = np.round(results)
    
    lst_accu_stratified_val.append(accuracy_score(y_test_fold, results))
    lst_f1_stratified_val.append(f1_score(results, y_test_fold, average='weighted') )
    lst_precision_stratified_val.append(precision_score(results, y_test_fold, average='weighted') )
    lst_recall_stratified_val.append(recall_score(results, y_test_fold, average='weighted') )
    fpr, tpr, thresholds = roc_curve(y_test_fold, results)
    lst_auc_stratified_val.append(auc(fpr, tpr))

    print("Evaluate on training data")
    results = sdp_classifier.evaluate([x_sparse_manifold_train, x_train_fold], y_train_fold)
    print("test loss, test acc:", results)

    print("Evaluate on test data")
    results = sdp_classifier.evaluate([x_sparse_manifold_test , x_test_fold], y_test_fold)
    print("test loss, test acc:", results)
    counter += 1
    

dict_test = {  "train_accu":lst_accu_stratified, 
               "train_precision":lst_precision_stratified, "train_recall":lst_recall_stratified,
               "train_auc":lst_auc_stratified, "train_f1":lst_f1_stratified,
             
               "test_accu":lst_accu_stratified_val, 
               "test_precision":lst_precision_stratified_val, "test_recall":lst_recall_stratified_val, 
               "test_auc":lst_auc_stratified_val, "test_f1":lst_f1_stratified_val,
               "Input": x_train_fold.shape[1], "RTE": x_sparse_manifold_train.shape[1]}

dict_test_result = pd.DataFrame.from_dict(dict_test)
dict_test_result.to_csv("hasil.csv")

with open('histories-tree-manifold-softlab.pickle'.format(counter), 'wb') as handle:
        pickle.dump(histories, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

/opt/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_isomap.py:384: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/opt/anaconda3/lib/python3.11/site-packages/scipy/sparse/_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Epoch 1/100

Epoch 1: val_loss improved from inf to 0.73075, saving model to sdp-tree-softlab-0.weights.best.keras
18/18 - 2s - loss: 0.7636 - accuracy: 0.5818 - val_loss: 0.7308 - val_accuracy: 0.4677 - 2s/epoch - 90ms/step
Epoch 2/100

Epoch 2: val_loss improved from 0.73075 to 0.63292, saving model to sdp-tree-softlab-0.weights.best.keras
18/18 - 0s - loss: 0.5732 - accuracy: 0.7145 - val_loss: 0.6329 - val_accuracy: 0.5161 - 124ms/epoch - 7ms/step
Epoch 3/100

Epoch 3: val_loss improved from 0.63292 to 0.59915, saving model to sdp-tree-softlab-0.weights.best.keras
18/18 - 0s - loss: 0.5180 - accuracy: 0.7491 - val_loss: 0.5991 - val_accuracy: 0.5161 - 104ms/epoch - 6ms/step
Epoch 4/100

Epoch 4: val_loss improved from 0.59915 to 0.58410, saving model to sdp-tree-softlab-0.weights.best.keras
18/18 - 0s - loss: 0.5045 - accuracy: 0.7509 - val_loss: 0.5841 - val_accuracy: 0.5484 - 108ms/epoch - 6ms/step
Epoch 5/100

Epoch 5: val_loss improved from 0.58410 to 0.56834, saving model to s

/opt/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_isomap.py:384: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/opt/anaconda3/lib/python3.11/site-packages/scipy/sparse/_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Epoch 1/100

Epoch 1: val_loss improved from inf to 0.55638, saving model to sdp-tree-softlab-1.weights.best.keras
18/18 - 2s - loss: 0.7440 - accuracy: 0.5782 - val_loss: 0.5564 - val_accuracy: 0.5161 - 2s/epoch - 87ms/step
Epoch 2/100

Epoch 2: val_loss improved from 0.55638 to 0.52845, saving model to sdp-tree-softlab-1.weights.best.keras
18/18 - 0s - loss: 0.5664 - accuracy: 0.7418 - val_loss: 0.5284 - val_accuracy: 0.5645 - 155ms/epoch - 9ms/step
Epoch 3/100

Epoch 3: val_loss did not improve from 0.52845
18/18 - 0s - loss: 0.5432 - accuracy: 0.7200 - val_loss: 0.5299 - val_accuracy: 0.5806 - 73ms/epoch - 4ms/step
Epoch 4/100

Epoch 4: val_loss did not improve from 0.52845
18/18 - 0s - loss: 0.5046 - accuracy: 0.7709 - val_loss: 0.5462 - val_accuracy: 0.5484 - 72ms/epoch - 4ms/step
Epoch 5/100

Epoch 5: val_loss did not improve from 0.52845
18/18 - 0s - loss: 0.4717 - accuracy: 0.7982 - val_loss: 0.5513 - val_accuracy: 0.5484 - 72ms/epoch - 4ms/step
Epoch 6/100

Epoch 6: val_loss 

/opt/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_isomap.py:384: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/opt/anaconda3/lib/python3.11/site-packages/scipy/sparse/_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


...

Epoch 1/100

Epoch 1: val_loss improved from inf to 0.51702, saving model to sdp-tree-softlab-7.weights.best.keras
18/18 - 2s - loss: 0.7404 - accuracy: 0.5945 - val_loss: 0.5170 - val_accuracy: 0.6129 - 2s/epoch - 91ms/step
Epoch 2/100

Epoch 2: val_loss improved from 0.51702 to 0.49477, saving model to sdp-tree-softlab-7.weights.best.keras
18/18 - 0s - loss: 0.5895 - accuracy: 0.6782 - val_loss: 0.4948 - val_accuracy: 0.6613 - 125ms/epoch - 7ms/step
Epoch 3/100

Epoch 3: val_loss improved from 0.49477 to 0.47609, saving model to sdp-tree-softlab-7.weights.best.keras
18/18 - 0s - loss: 0.5457 - accuracy: 0.7255 - val_loss: 0.4761 - val_accuracy: 0.7097 - 120ms/epoch - 7ms/step
Epoch 4/100

Epoch 4: val_loss did not improve from 0.47609
18/18 - 0s - loss: 0.5094 - accuracy: 0.7491 - val_loss: 0.4960 - val_accuracy: 0.6774 - 69ms/epoch - 4ms/step
Epoch 5/100

Epoch 5: val_loss did not improve from 0.47609
18/18 - 0s - loss: 0.4669 - accuracy: 0.7673 - val_loss: 0.5043 - val_accuracy: 0.

/opt/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_isomap.py:384: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/opt/anaconda3/lib/python3.11/site-packages/scipy/sparse/_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Epoch 1/100

Epoch 1: val_loss improved from inf to 0.52833, saving model to sdp-tree-softlab-8.weights.best.keras
18/18 - 1s - loss: 0.8669 - accuracy: 0.5273 - val_loss: 0.5283 - val_accuracy: 0.6452 - 1s/epoch - 83ms/step
Epoch 2/100

Epoch 2: val_loss improved from 0.52833 to 0.50515, saving model to sdp-tree-softlab-8.weights.best.keras
18/18 - 0s - loss: 0.6077 - accuracy: 0.6727 - val_loss: 0.5052 - val_accuracy: 0.6290 - 129ms/epoch - 7ms/step
Epoch 3/100

Epoch 3: val_loss did not improve from 0.50515
18/18 - 0s - loss: 0.5225 - accuracy: 0.7473 - val_loss: 0.5123 - val_accuracy: 0.6290 - 73ms/epoch - 4ms/step
Epoch 4/100

Epoch 4: val_loss did not improve from 0.50515
18/18 - 0s - loss: 0.5326 - accuracy: 0.7545 - val_loss: 0.5061 - val_accuracy: 0.6290 - 66ms/epoch - 4ms/step
Epoch 5/100

Epoch 5: val_loss improved from 0.50515 to 0.49730, saving model to sdp-tree-softlab-8.weights.best.keras
18/18 - 0s - loss: 0.5121 - accuracy: 0.7691 - val_loss: 0.4973 - val_accuracy: 0.6

/opt/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_isomap.py:384: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/opt/anaconda3/lib/python3.11/site-packages/scipy/sparse/_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Epoch 1/100

Epoch 1: val_loss improved from inf to 0.45962, saving model to sdp-tree-softlab-9.weights.best.keras
18/18 - 1s - loss: 0.7530 - accuracy: 0.5855 - val_loss: 0.4596 - val_accuracy: 0.7097 - 1s/epoch - 82ms/step
Epoch 2/100

Epoch 2: val_loss did not improve from 0.45962
18/18 - 0s - loss: 0.6000 - accuracy: 0.6745 - val_loss: 0.5028 - val_accuracy: 0.6452 - 70ms/epoch - 4ms/step
Epoch 3/100

Epoch 3: val_loss did not improve from 0.45962
18/18 - 0s - loss: 0.5091 - accuracy: 0.7418 - val_loss: 0.5128 - val_accuracy: 0.6613 - 67ms/epoch - 4ms/step
Epoch 4/100

Epoch 4: val_loss did not improve from 0.45962
18/18 - 0s - loss: 0.5250 - accuracy: 0.7509 - val_loss: 0.5092 - val_accuracy: 0.6613 - 85ms/epoch - 5ms/step
Epoch 5/100

Epoch 5: val_loss did not improve from 0.45962
18/18 - 0s - loss: 0.4653 - accuracy: 0.7873 - val_loss: 0.5100 - val_accuracy: 0.6452 - 101ms/epoch - 6ms/step
Epoch 6/100

Epoch 6: val_loss did not improve from 0.45962
18/18 - 0s - loss: 0.4727 - ac

In [94]:
dict_test_result

,train_accu,train_precision,train_recall,train_auc,train_f1,test_accu,test_precision,test_recall,test_auc,test_f1,Input,RTE
0,0.975490,0.975923,0.975490,0.975490,0.975485,0.941176,0.942907,0.941176,0.941176,0.941227,29,10
1,0.959150,0.959412,0.959150,0.959150,0.959145,0.852941,0.852941,0.852941,0.852941,0.852941,29,10
2,0.972222,0.972270,0.972222,0.972222,0.972222,0.941176,0.942907,0.941176,0.941176,0.941227,29,10
3,0.965686,0.965692,0.965686,0.965686,0.965686,0.897059,0.900952,0.897059,0.897059,0.897260,29,10
4,0.975490,0.975496,0.975490,0.975490,0.975490,0.926471,0.937284,0.926471,0.926471,0.926870,29,10
5,0.972222,0.972270,0.972222,0.972222,0.972222,0.911765,0.913495,0.911765,0.911765,0.911841,29,10
6,0.968954,0.969002,0.968954,0.968954,0.968954,0.897059,0.907872,0.897059,0.897059,0.897618,29,10
7,0.954248,0.954441,0.954248,0.954248,0.954244,0.882353,0.882353,0.882353,0.882353,0.882353,29,10
8,0.954248,0.954334,0.954248,0.954248,0.954246,0.911765,0.913495,0.911765,0.911765,0.911841,29,10
9,0.970588,0.970930,0.970588,0.970588,0.970583,0.897059,0.897491,0.897059,0.897059,0.897081,29,10


In [107]:
best = 4

dbfile = open('rte-tree-softlab-{0}.pickle'.format(best), 'rb')    
rte = pickle.load(dbfile)
dbfile.close()

dbfile = open('manifold-tree-softlab-{0}.pickle'.format(best), 'rb')    
manifold = pickle.load(dbfile)
dbfile.close()

dbfile = open('scaler-tree-softlab-{0}.pickle'.format(best), 'rb')    
scaler2 = pickle.load(dbfile)
dbfile.close()

dbfile = open('sdp-classifier-tree-softlab-{0}.pickle'.format(best), 'rb')    
sdp_classifier = pickle.load(dbfile)
dbfile.close()
sdp_classifier.load_weights('sdp-tree-softlab-{0}.weights.best.keras'.format(best))

In [108]:
X_sparse_embedding = rte.transform(X_norm_scale)
X_sparse_manifold = manifold.transform(X_sparse_embedding)
X_sparse_manifold = scaler2.transform(X_sparse_manifold)
X_sparse_embedding.shape

(392, 108058)

In [109]:
X_norm_scale.shape, df.shape

((392, 29), (392, 30))

In [110]:
print("Evaluate on training data")
results = sdp_classifier.evaluate([X_sparse_manifold, X_norm_scale], y, batch_size=128)
print("test loss, test acc:", results)

Evaluate on training data
4/4 [==============================] - 0s 3ms/step - loss: 0.1429 - accuracy: 0.9592
test loss, test acc: [0.14294999837875366, 0.9591836929321289]


In [111]:
from sklearn.metrics import classification_report

results = sdp_classifier.predict([X_sparse_manifold,X_norm_scale])
results=np.round(results)
print(classification_report(y,results))

13/13 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

         0.0       0.99      0.96      0.98       340
         1.0       0.79      0.94      0.86        52

    accuracy                           0.96       392
   macro avg       0.89      0.95      0.92       392
weighted avg       0.96      0.96      0.96       392



In [112]:
print(y.shape, results.shape)
fpr, tpr, thresholds = roc_curve(y, results)
print(auc(fpr, tpr))

(392,) (392, 1)
0.9520361990950227


In [115]:
list_result = []

for label in my_dataset:
    dt = adjustdataset(my_dataset[label], df)
    pt, scaler, imputer, X_norm_scale_dt, y_dt = preprocessing(dt,imputer,pt, scaler, show=False)
    x_sparse_embedding_dt = rte.transform(X_norm_scale_dt)
    x_sparse_manifold_dt = manifold.transform(x_sparse_embedding_dt)
    x_sparse_manifold_dt = scaler2.transform(x_sparse_manifold_dt)
    results = sdp_classifier.predict([x_sparse_manifold_dt,X_norm_scale_dt])
    result  = np.round(results)
    fpr, tpr, thresholds = roc_curve(y_dt, result)
    list_result.append({'label':label, 'accuracy' : accuracy_score(y_dt, result),'precision':precision_score(y_dt, result, average='weighted'),
                  'recall': recall_score(y_dt, result, average='weighted'), 'auc':auc(fpr, tpr),
                        'f1-score':f1_score(y_dt, result, average='weighted')})
    #heat_map(y_dt,result,label)


    

16/16 [==============================] - 0s 1ms/step


In [116]:
df_result = pd.DataFrame(list_result)
df_result

,label,accuracy,precision,recall,auc,f1-score
0,ar1,0.957115,0.963415,0.957115,0.947302,0.958985
1,ar3,0.962637,0.967216,0.962637,0.957278,0.963942
2,ar4,0.961924,0.966639,0.961924,0.960431,0.963206
3,ar5,0.948598,0.952358,0.948598,0.921286,0.949922
4,ar6,0.955375,0.960319,0.955375,0.942734,0.956890


In [117]:
df_result

,label,accuracy,precision,recall,auc,f1-score
0,ar1,0.957115,0.963415,0.957115,0.947302,0.958985
1,ar3,0.962637,0.967216,0.962637,0.957278,0.963942
2,ar4,0.961924,0.966639,0.961924,0.960431,0.963206
3,ar5,0.948598,0.952358,0.948598,0.921286,0.949922
4,ar6,0.955375,0.960319,0.955375,0.942734,0.956890


In [103]:
df_result[['accuracy','precision','recall','auc','f1-score']].mean()

accuracy     0.944538
precision    0.949556
recall       0.944538
auc          0.915050
f1-score     0.946268
dtype: float64

In [104]:
sdp_classifier.summary()

Model: "model_23"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 X_train1 (InputLayer)       [(None, 10)]                 0         []                            
                                                                                                  
 X_train2 (InputLayer)       [(None, 29)]                 0         []                            
                                                                                                  
 X_train1_layer (Dense)      (None, 64)                   704       ['X_train1[0][0]']            
                                                                                                  
 X_train2_layer (Dense)      (None, 64)                   1920      ['X_train2[0][0]']            
                                                                                           

In [105]:
list_result = []

for label in my_dataset:
    dt = my_dataset[label]
    list_result.append({'Dataset':label, 'Instances':dt.shape[0], 'Features':dt.shape[1],
                        'Defective Instances': dt[dt['Defective'] == 'Y'].shape[0],
                        'Non-Defective Instances': dt[dt['Defective'] == 'N'].shape[0],})


    

In [106]:
df_result = pd.DataFrame(list_result)
df_result

,Dataset,Instances,Features,Defective Instances,Non-Defective Instances
0,ar1,121,30,9,112
1,ar3,63,30,8,55
2,ar4,107,30,20,87
3,ar5,36,30,8,28
4,ar6,101,30,15,86
